# http://www.cnblogs.com/charlotte77/p/5366578.html

# 先jieba分词，再进行one-hot encoding（数值由tf-idf来确定），最后进行AE+聚类

# Feature Engineering

In [1]:
import pandas as pd
data_all_docs=pd.read_csv('train_set.csv')

FileNotFoundError: File b'train_set.csv' does not exist

In [ ]:
data_all_docs.head()

In [ ]:
# data_all_docs.head()
data=data_all_docs[data_all_docs['class']<3]
# data.iloc[:,3].shape
data=data[:1000]

In [ ]:
# def sljoin(sl):return x,end=' '.join(eval(sl))

from sklearn.feature_extraction.text import TfidfVectorizer
corpus=data['word_seg']
vectorizer = TfidfVectorizer()
weights= vectorizer.fit_transform(corpus).toarray()
# words=vectorizer.get_feature_names()
wordsweight=pd.DataFrame(weights)

# AE

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

class AutoEncoder():
    """ Auto Encoder  
 11     layer      1     2    ...    ...    L-1    L
 12       W        0     1    ...    ...    L-2
 13       B        0     1    ...    ...    L-2
 14       Z              0     1     ...    L-3    L-2
 15       A              0     1     ...    L-3    L-2
 16     """
    
    def __init__(self, X, Y, nNodes):
        # training samples
        self.X = X
        self.Y = Y
        # number of samples
        self.M = len(self.X)
        # layers of networks
        self.nLayers = len(nNodes)
        # nodes at layers
        self.nNodes = nNodes
        # parameters of networks
        self.W = list()
        self.B = list()
        self.dW = list()
        self.dB = list()
        self.A = list()
        self.Z = list()
        self.xunlian=[]
        self.xunlian2=[]
        self.delta = list()
        for iLayer in range(self.nLayers - 1):
            self.W.append( np.random.rand(nNodes[iLayer]*nNodes[iLayer+1]).reshape(nNodes[iLayer],nNodes[iLayer+1]) ) 
            self.B.append( np.random.rand(nNodes[iLayer+1]) )
            self.dW.append( np.zeros([nNodes[iLayer], nNodes[iLayer+1]]) )
            self.dB.append( np.zeros(nNodes[iLayer+1]) )
            self.A.append( np.zeros(nNodes[iLayer+1]) )
            self.Z.append( np.zeros(nNodes[iLayer+1]) )
            self.delta.append( np.zeros(nNodes[iLayer+1]) )
             
         # value of cost function
        self.Jw = 0.0
        # active function (logistic function)
        self.sigmod = lambda z: 1.0 / (1.0 + np.exp(-z))
        # learning rate 1.2
        self.alpha = 2.5
        # steps of iteration 30000
        self.steps = 2
        
    def BackPropAlgorithm(self):
        # clear values
        self.Jw -= self.Jw
        for iLayer in range(self.nLayers-1):
            self.dW[iLayer] -= self.dW[iLayer]
            self.dB[iLayer] -= self.dB[iLayer]
         # propagation (iteration over M samples)    
        for i in range(self.M):
             # Forward propagation
            for iLayer in range(self.nLayers - 1):
                if iLayer==0: # first layer
                    self.Z[iLayer] = np.dot(self.X[i], self.W[iLayer])
                else:
                    self.Z[iLayer] = np.dot(self.A[iLayer-1], self.W[iLayer])
                self.A[iLayer] = self.sigmod(self.Z[iLayer] + self.B[iLayer])            
             # Back propagation
            for iLayer in range(self.nLayers - 1)[::-1]: # reserve
                if iLayer==self.nLayers-2:# last layer
                    self.delta[iLayer] = -(self.X[i] - self.A[iLayer]) * (self.A[iLayer]*(1-self.A[iLayer]))
                    self.Jw += np.dot(self.Y[i] - self.A[iLayer], self.Y[i] - self.A[iLayer])/self.M
                else:
                    self.delta[iLayer] = np.dot(self.W[iLayer].T, self.delta[iLayer+1]) * (self.A[iLayer]*(1-self.A[iLayer]))
                # calculate dW and dB 
                if iLayer==0:
                    self.dW[iLayer] += self.X[i][:, np.newaxis] * self.delta[iLayer][:, np.newaxis].T
                else:
                    self.dW[iLayer] += self.A[iLayer-1][:, np.newaxis] * self.delta[iLayer][:, np.newaxis].T
                self.dB[iLayer] += self.delta[iLayer] 
         # update
                for iLayer in range(self.nLayers-1):
                    self.W[iLayer] -= (self.alpha/self.M)*self.dW[iLayer]
                    self.B[iLayer] -= (self.alpha/self.M)*self.dB[iLayer]
        
    def PlainAutoEncoder(self):
        for i in range(self.steps):
            self.BackPropAlgorithm()
            print ("step:%d" % i, "Jw=%f" % self.Jw)
    def ValidateAutoEncoder(self):
        for i in range(self.M):
            print (self.X[i])
            for iLayer in range(self.nLayers - 1):
                if iLayer==0: # input layer
                    self.Z[iLayer] = np.dot(self.X[i], self.W[iLayer])
                    
                else:
                    self.Z[iLayer] = np.dot(self.A[iLayer-1], self.W[iLayer])
                self.A[iLayer] = self.sigmod(self.Z[iLayer] + self.B[iLayer])
                print ("\t layer=%d" % iLayer, self.A[iLayer])
            self.xunlian.append(self.A[1])
            self.xunlian2.append(self.A[0])
        return self.xunlian2


In [3]:
import numpy as np
x = np.array(wordsweight)

xx = preprocessing.scale(x)

NameError: name 'wordsweight' is not defined

In [ ]:
len(xx[0])

In [ ]:
nNodes = np.array([63759,1000,63759])
ae3 = AutoEncoder(xx,xx,nNodes)
# len(nNodes)
len(xx)

In [ ]:

ae3.PlainAutoEncoder()
wordsweight2,wordsweight3=ae3.ValidateAutoEncoder()
len(wordsweight2)

In [ ]:
wordsweight3=ae3.ValidateAutoEncoder()

# K-MEANS

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=2, random_state=0)

In [ ]:
k_means.fit(wordsweight2)
asa=k_means.predict(wordsweight2)

In [ ]:
k_means.fit(wordsweight3)
asa=k_means.predict(wordsweight3)